# Altering an Existing Model
Since my attempts to train the model only really got 70% accuracy at most, and the graphs indicated under/over fitting depending on my attempt, I decided to use [this model from Kaggle](https://www.kaggle.com/code/muhammadfaizan65/plant-species-classification-vgg16/output) instead.

By using this model, I could modify it to return the predicitons I expect while still maintaining good accuracy for my dataset

In [4]:
from tensorflow.keras.models import load_model
import tensorflow as tf

# Load the .keras model from Kaggle
model = load_model("model/best_model.keras")

# Load dataset
dataset_path = "house_plant_species"

# only keep supported formats like .jpg, .jpeg, .png
valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']

ds_train_ = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    labels='inferred',
    label_mode='categorical',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)

# Check the class names
class_names = ds_train_.class_names

Found 13724 files belonging to 47 classes.


### Modifications
To ensure the model can return the label specified for my front end, I will slightly modify it and ensure it is saved as a `SavedModel` instead of `.keras` as TensorFlow Serving requires the first one.

In [5]:
class LabeledModel(tf.keras.Model):
    def __init__(self, base_model, class_names):
        super().__init__()
        self.base_model = base_model
        self.class_names = tf.constant(class_names)

    def call(self, inputs):
        probabilities = self.base_model(inputs)  # Raw predictions
        predicted_indices = tf.argmax(probabilities, axis=-1)  # Predicted indices
        predicted_labels = tf.gather(self.class_names, predicted_indices)  # Class names
        return {"probabilities": probabilities, "labels": predicted_labels}

# Wrap the model
labeled_model = LabeledModel(model, class_names)

# Build the model by passing an example input
example_input = tf.random.normal([1, 128, 128, 3])  # Example input shape (batch_size, height, width, channels)
labeled_model(example_input)  # This triggers the building of the model

# Now export the labeled model
labeled_model.export("model/plant_model/3")

I0000 00:00:1735071537.495027    3677 cuda_dnn.cc:529] Loaded cuDNN version 90300


INFO:tensorflow:Assets written to: model/plant_model/3/assets


INFO:tensorflow:Assets written to: model/plant_model/3/assets


Saved artifact at 'model/plant_model/3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None)
Output Type:
  Dict[['probabilities', TensorSpec(shape=(None, 47), dtype=tf.float32, name=None)], ['labels', TensorSpec(shape=(None,), dtype=tf.string, name=None)]]
Captures:
  139833872850032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139833872854960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139833872858656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139833872863952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139833872870640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139833872867472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139833872872576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139833872871168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139833872873456: TensorSpec(shape=(), dtype=tf.resource, name=None)


### Conclusion
And that's it. Most of the work was done thanks to [Muhammad Faizan on Kaggle](https://www.kaggle.com/code/muhammadfaizan65/plant-species-classification-vgg16/notebook) and I just modified what I needed. I hope to take this experience and learn what I did wrong. Now, the app returns *mostly* correct predictions.